В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

### Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (более строго говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeigborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [34]:
from sklearn import datasets, ensemble
import numpy as np
from matplotlib import pyplot as plt

In [35]:
dataset = datasets.load_digits()
X = dataset['data']
y = dataset['target']

In [36]:
num_elements = X.shape[0]
index = int(0.25*num_elements)

X_train = X[:-index]
X_test = X[-index:]

y_train = y[:-index]
y_test = y[-index:]

In [37]:
X_train.shape, X_test.shape

((1348, 64), (449, 64))

In [38]:
y_train.shape, y_test.shape

((1348,), (449,))

In [39]:
x = X_test[0, :]

In [52]:
def determine_knn(x, X, y):
    distances = ((X - x) ** 2).sum(axis = 1)
    min_d = distances[0]
    min_t = y[0]
    for d, t in zip(distances, y):
        if d < min_d:
            min_d = d
            min_t = t
    return min_t

In [53]:
predictions = []
for i in range(X_test.shape[0]):
    x = X_test[i, :]
    predictions.append(determine_knn(x, X_train, y_train))

In [62]:
predictions = np.array(predictions)

In [63]:
accuracy = float((y_test != predictions).sum())/y_test.shape[0]
accuracy

0.0378619153674833

### Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [65]:
clf = ensemble.RandomForestClassifier(n_estimators = 1000)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [66]:
predictions = clf.predict(X_test)

In [69]:
accuracy = float((y_test != predictions).sum())/y_test.shape[0]
accuracy

0.0645879732739421